In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import os
import sys

In [13]:
sys.path.append(os.path.abspath(os.path.join("../..")))

In [14]:
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz

from notebooks.utils.constants import (
    AvailableEmbeddingModels,
    AvailableModels,
    question_ids_map,
)
from notebooks.utils.process_lease import process_lease
from notebooks.utils.questions import (
    construct_single_question_for_ai,
)

## Threshold for Levenshtein distance


In [15]:
threshold = 80

Retrieve questions


In [16]:
questions_df = pd.read_excel("../../data/asc_842/grt_ai_reader_questions_v1.xlsx")
questions_df = questions_df.drop(["encoding"], axis=1)
questions_df = questions_df.replace({np.nan: None})

Retrieve answers file


In [17]:
answers_df = pd.read_excel("../../data/asc_842/lease_agreements_info/answers.xlsx")

Get questions to send to the AI


In [18]:
question_id = question_ids_map["lessor_name"]
question_df_filtered = questions_df[questions_df["id"] == question_id]
question_as_json = question_df_filtered.to_dict(orient="records")[0]
question_for_ai = construct_single_question_for_ai(question_as_json)

In [19]:
file_name = "lease001"
local_file = f"../../data/asc_842/lease_agreements/{file_name}.pdf"
file_key = f"eafit/{file_name}.pdf"
namespace = f"eafit_{file_name}"

In [20]:
model: AvailableModels = "gpt-3.5-turbo"
embedding_model: AvailableEmbeddingModels = "text-embedding-3-large"

In [21]:
result = process_lease(
    file_name=file_name,
    namespace=namespace,
    question_for_ai=question_for_ai,
    question_id=question_id,
    answers_df=answers_df,
    model=model,
    embedding_model=embedding_model,
    index_name="eafit-text-embedding-3-large",
)

In [22]:
answer = result["answer"]
real_answer_unprocessed = result["real_answer_unprocessed"]

In [23]:
answer

['Kyle J. Beller']

In [24]:
similarity_score = fuzz.ratio(
    answer[0].lower(), real_answer_unprocessed.iloc[0].lower()
)

In [25]:
similarity_score >= threshold

False